## Overview

This script will move downloaded images to `prepare_dataset`. The process is briefed as follows:

- Images downloaded from `notebooks/download.ipynb` will be stored in `download_file` folder.
- Under `download_file` folder, images will be grouped according to the polygon they belong to.
- In each polygon, 2 types of images are present: geotiff with band information, and a mask file.
- Geotiffs should be moved and stored in `prepare_dataset/images_directory{group_id}` folder.
- Mask files are stored in `prepare_dataset/mask_directory{group_id}` folder.

## Implementation

In [1]:
import os
import glob
import shutil


In [4]:
def fast_scandir(dirname: str) -> list:
    """
    Scan and return all subfolders of a directory.
    """
    subfolders= [f.path for f in os.scandir(dirname) if f.is_dir()]
    for dirname in list(subfolders):
        subfolders.extend(fast_scandir(dirname))
    return subfolders

source_path = "../download_file/"
subfolders = fast_scandir(source_path)
print(subfolders, sep="\n")


['../download_file/S2A_7VFF_20230820_0_L2A', '../download_file/S2B_7VFG_20230822_0_L2A', '../download_file/S2A_7VFG_20230820_0_L2A', '../download_file/S2B_8VLM_20230822_0_L2A', '../download_file/S2B_8VML_20230822_0_L2A', '../download_file/S2A_7VFF_20230820_0_L2A/20230820', '../download_file/S2B_7VFG_20230822_0_L2A/20230822', '../download_file/S2A_7VFG_20230820_0_L2A/20230820', '../download_file/S2A_7VFG_20230820_0_L2A/20230820/tiles', '../download_file/S2A_7VFG_20230820_0_L2A/20230820/tiles/9_4', '../download_file/S2A_7VFG_20230820_0_L2A/20230820/tiles/7_2', '../download_file/S2A_7VFG_20230820_0_L2A/20230820/tiles/5_8', '../download_file/S2A_7VFG_20230820_0_L2A/20230820/tiles/10_6', '../download_file/S2A_7VFG_20230820_0_L2A/20230820/tiles/5_1', '../download_file/S2A_7VFG_20230820_0_L2A/20230820/tiles/5_6', '../download_file/S2A_7VFG_20230820_0_L2A/20230820/tiles/9_2', '../download_file/S2A_7VFG_20230820_0_L2A/20230820/tiles/9_5', '../download_file/S2A_7VFG_20230820_0_L2A/20230820/tiles